It might be useful to run 'pip3 install &lt;library&gt; --upgrade" to make sure you have latest versions of pandas, matplotlib, folium, etc.

In [ ]:
# load libraries

import folium
from folium.plugins import MarkerCluster
import pandas as pd    # for working with "rows/columns" oriented data
import pymysql.cursors # 

# not using these right now, but I might...
import io  # useful routines for input/output

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
         
import numpy as np



In [ ]:
# folium.Map 

# use location to center the map. A bigger zoom # zooms in. try it.

med_map = folium.Map(location=[41.9028, 12.4964],zoom_start=4,tiles='cartodbpositron')
med_map

In [ ]:
# https://leaflet-extras.github.io/leaflet-providers/preview/
# go the the above URL to see a variety of basemaps.
# when you try one, you should see the URL to use for tileset.
# you sometimes need to adjust that. particularly change '{ext} to just '.png'.

tileset = r'http://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}.png'
tiled = folium.Map(location=[41.9028, 12.4964], zoom_start=4,
                 tiles=tileset, attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')
tiled

In [ ]:
# i will email everyone this file. put it in same folder
# Connect to the database
db_params = open("sebastia_adsqro_params.txt").read().split()

In [ ]:
# Connect to the database

connection = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [ ]:
# Noting that some are having problems with pd.read_sql, I'm stick with pymysql.
# But I move that quickly into a pandas datasframe.
# Try that with your more complex queries. Does that help?

sql = "select * from ramphs"


# this is a pretty efficient code block for going from cursor to dataframe
with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

# df # uncommanet at start of line if you want to see contents

In [ ]:
tileset = r'http://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}.png'
ramphs = folium.Map(location=[41.9028, 12.4964], zoom_start=4,
                 tiles=tileset, attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

# iterrows() is useful here. That moves through dataframe row by row
# row is a dictionary so we pass its values to each call to folium.Marker()


for index,row in df[['label','latitude','longitude']].iterrows():
    # built a marker and then add it to map with add_to()
    folium.Marker([row['latitude'],row['longitude']], popup=row['label']).add_to(ramphs)


# might take a little while to display. the loop above is slow.
ramphs

In [ ]:
# here I'm calling folium.CircleMarker insead of folium.Marker
# circles don't adjust when you zoom in.
# experiment with radius size.

tileset = r'http://{s}.tiles.wmflabs.org/bw-mapnik/{z}/{x}/{y}.png'

ramphs = folium.Map(location=[41.9028, 12.4964], zoom_start=4,
                 tiles=tileset, attr='&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

for index,row in df[['label','latitude','longitude']].iterrows():
    
    folium.CircleMarker([row['latitude'],row['longitude']],
                        radius = 1000,
                        popup=row['label'],
                        color = 'red').add_to(ramphs)

ramphs

In [ ]:
# goal is to color amphitheater by capacity. with black for null values.

cuts  = pd.cut(df['capacity'],bins = 5, labels = False, right = True)
# after this, lowest value will be 0, highest 4. For a total of 5 bins.

# shift values so they are 1 to 5.
cuts += 1 # this syntax updates the dataframe in place, which can be useful.

# fill nulls with 0.
cuts.fillna(0,inplace = True) # again, I'm doing this in place


# pd.concat with axis = 1 combines dataframes side-by-side
ramphs_cuts = pd.concat([df,pd.DataFrame({"cut":cuts})], axis = 1)

# ramphs_cuts # uncomment to see that cuts now at right side of resulting dataframe.


In [ ]:
# a color ramp from green to red (though with the first component as black)
colors = ["#000000", "#00ff00", "#50a000", "#bfbf00", "#df7000", "#ff0000"]

# trying a different basemap
tileset = r'http://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}'

ramphs_colors = folium.Map(location=[41.9028, 12.4964], zoom_start=4,
                 tiles=tileset, attr='Tiles &copy; Esri &mdash; Source: USGS, Esri, TANA, DeLorme, and NPS')

# key construct is "colors[int(row['cut'])]" . Use the value of cut in each row to select
# a value from the colors python list.
# and see that I'm selecting three columns. That's unnecessary but keeps me focused.
for index,row in ramphs_cuts[['label','latitude','longitude', 'cut']].iterrows():
    
    folium.CircleMarker([row['latitude'],row['longitude']],
                        radius = 500,
                        popup=(row['label'] + ' ' + str(row['cut'])),
                        fill_color = colors[int(row['cut'])],
                        color = colors[int(row['cut'])]).add_to(ramphs_colors)

ramphs_colors

In [ ]:
# clusters can be useful.

tileset = 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/tile/{z}/{y}/{x}'
      
clustered = folium.Map(location=[41.9028, 12.4964], zoom_start=4,
                 tiles=tileset, attr='&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

# create two empty lists
popups, locations = [], []

# use iterrows() to loop through dataframe and build popups and locations
for index,row in df[['label','latitude','longitude']].iterrows():
    locations.append([row['latitude'],row['longitude']])
    popups.append(row['label'])


# pass locations and popups to MarkerCluster.
clustered.add_children(MarkerCluster(locations=locations, popups=popups))



clustered

In [ ]:
# connection.commit()
# connection.close()